In [1]:
#| default_exp ECMs.episodic

In [2]:
#| export
import numpy as np
import scipy
from projective_simulation.ECMs.abstract_ECM import ECM
from projective_simulation.utils import _softmax

class Episodic_Memory(ECM):
    def __init__(self,
                 num_actions: int,
                 capacity: int = 10,
                 softmax: float = 0.7,
                 focus: float = 1.,
                 error_tolerance: float = 0.01,
                 min_expectation: float = 0.01,
                 deliberation_length: int = 1,
                 t: int = 0,
                ):
        '''
        The episodic memory ECM stores percept information in a time ordered system of memory traces by establishing connections (trace_encoder) with weights (hmatrix) between excited perceptual representations and an excited memory trace.
        The agent maintains a belief state through the activation (different from excitation) of memory traces, where the strength of activation reflects the strength of the agent's belief that its current state-in-the-world is effectively the same as a state-in-the-world represented by that memory trace
        The agent also maintains an expectation state by priming perceptual representations as a function of (1) its belief state, (2) connections between memory traces (mmatrix, stricly time-ordered in this implementation), and (3) the h-matrix
        The difference between the agent's expectation state and the excitation of perceptual represenations in the next time step is computed as a surprise and assigned to the last excited memory trace as a valence state
        Perceptual representations are excited as a function of sensory-motor interactions with the world and the agent's expectation state
        Memory Traces are excited in a time-ordered, winner-takes all fashion.
        The agent's beliefs are updated by a process of deliberation, in which excited perceptual representations are activated and those activations then diffuse across the ECM as if they were caused by particles performing a random walk on the ECM graph
        The probability that an activation particle walks from one node to another is a function of the edge weights in the hmatrix connected to its current node (perceptual representation or memory trace), and the belief states and valence states of all nodes to which those edges lead
        The focus parameter of the Episodic ECM adds weight to a single departing edge from each node, adding stochastic bias to the random walk. Rewards reinforce edge weights in the ECM proportionally to the difference between the expected particle mass to traverse that edge and the observed mass
        Rewards are determined as a function of the "surprise advantage", i.e. how much more or less surprised the agent was then on average over its previous (relevant) experience.

        inputs
        num_actions: the number of perceptual representations available to the agent which, when excited, have an effect on the agent's environment. Action representations are primed differently than sensory representations and do not affect surprise
        capacity: the number of memory traces avaiable to the agent. If simulation time exceeds capacity, the oldest memory trace will be overwritten each time step
        softmax: to determine random walk probailities, edge weights are normalized using a softmax function with this variable as the temperature constant
        t: used to model the temporal excitation sequence of memory traces
        focus:  Focus scales the effect of stochastic processes underlying random walks on the ECM. (i.e. it scales the effect of Projective Simulation)
                Think of deliberation in an Episodic ECM like a very large number of particles diffusing on the ECM graph, but at each node a single particle is chosen at random and some proportion of particles are pulled along with that one. Focus defines that proportion
                If focus == 1, deliberation acts as a random walk of a single (massive) particle on the ECM. 
                If focus == 0, deliberation acts as the diffusion of a large (approaching infinite) number of particles on the ECM
        error_tolerance: How strongly the agent discounts the similarity between two states per bit of mismatched sensory information when establishing a new belief state
        min_expectation: a baseline value for the priming of perceptual representations. Must be greater than 0 to prevent infinate surprise if a perceptual representation is excited that was not predicted by the agents belief state
        deliberation_length: the number of diffusive steps in the agent's deliberations
        '''

        #initialize constants
        self.num_actions = num_actions
        self.capacity = capacity
        self.softmax = 0.7 
        self.focus = focus 
        self.error_tolerance = error_tolerance
        self.min_expectation = min_expectation
        self.deliberation_length = deliberation_length
        self.surprise: float = None
        
        #initialize modelling variables
        self.t = t

        #initialize ECM states
        self.hmatrix = np.zeros((self.num_actions, self.capacity)) #weights connecting percept nodes to trace nodes
        self.mmatrix = np.zeros((self.capacity, self.capacity)) #initialize with no connections between memory traces
        self.trace_encoder = np.zeros((self.num_actions, self.capacity), dtype = 'bool') #boolean matrix indicating whether a percept node was encoded in a trace
        self.action_encoder = np.ones(self.num_actions, dtype = 'bool') #boolean matrix indicated whether a percept node belongs to an action
        self.expectations = np.zeros(self.num_actions) + self.min_expectation
        self.beliefs = np.zeros(self.capacity) #initilize with no belief weight on any memory traces
        self.valences = np.array([np.nan] * self.capacity) #use nan here because we will want to takes means of this array as it is filled
        self.percept_activations = np.zeros(np.shape(self.hmatrix)[0]) #initialize with no activation of percept nodes
        self.trace_activations = np.zeros(self.capacity) #initialize with no activation of memory traces
        self.trace_excitations = np.zeros(self.capacity) #trace excitations represent the sensory evidence that the world is currently in a state that is effectively represented by the excited trace
        

    def deliberate(self, percept):
        self.add_percept(percept)
        self.surprise = self.get_surprise(percept)
        self.excite_traces(percept)
        self.encode_trace(percept)
        self.activate()
        for deliberation_step in range(self.deliberation_length):
            self.diffuse_activation()
        self.predict()
        self.t = (self.t + 1) % self.capacity
        return(self.expectations[self.action_encoder]) #return action priming

    def add_percept(self, percept):
        if len(percept) > np.shape(self.hmatrix)[0]: #if percept is longer than the first dimension of the hmatrix it means there is something new in the observation (handled by preprocessor)
            i = np.shape(self.hmatrix)[0] #get index for new elements in ECM
            new_elements = percept[i:len(percept)]    
            self.hmatrix = np.append(self.hmatrix, np.zeros((len(new_elements), self.capacity)), axis = 0) #add baseline weights to connections between new percept nodes and all traces
            self.trace_encoder = np.append(self.trace_encoder, np.zeros((len(new_elements), self.capacity), dtype = 'bool'), axis = 0) #new percept nodes have no existing connections to trace nodes
            self.action_encoder = np.append(self.action_encoder, np.zeros(len(new_elements), dtype = 'bool')) #this ECM does not support new actions, so new percept nodes are sensory by default
            self.expectations = np.append(self.expectations, np.zeros(len(new_elements)) + self.min_expectation)

    def excite_traces(self, percept):
        self.trace_excitations = np.zeros(self.capacity)
        # each trace is excited proportionally to the probability that of the n connections it has to perceptual representations, r of those perceptual representations are excited . . .
        # . . . if that trace effectively represents the current world state . . .
        # . . . and sensory_error gives the probability that a connected sensory_representation fails to excite when the world is in the state represented by the memory trace.
        # this probability is given by the binomial distribution
        for t_index in range(self.capacity):
            n = np.sum(self.trace_encoder[0:,t_index])
            if n > 0: #no excitation if trace has no connections (functionally this shouldn't matter, but it is nicer for interpretation)
                r = np.sum([self.trace_encoder[p_index,t_index] and percept[p_index] for p_index in range(len(percept))]) # for each percept, checks that it is both excited and connected to trace, then counts
                self.trace_excitations[t_index] = scipy.stats.binom.pmf(r,n,p = 1-self.error_tolerance)
    
    def encode_trace(self, percept):
        self.trace_encoder[0:,self.t] = [x > 0 for x in percept] #connect all excited percepts to current trace
        self.mmatrix[self.t-1,self.t] = 1 #create forward connection from previous trace to current trace (this is a spurious but inconsequential connection for an agent's first step)
        self.mmatrix[self.t,0:] = [0 for x in self.mmatrix[self.t,0:]] #break any forward connections from current trace (only relevant if t is greater than self.capacity)
        self.valences[self.t-1] = self.surprise #valence is assigned to the last trace, it relfects how surprised the agent was by the outcome of the action it took in that trace
    
    def activate(self):
        activation_weights = _softmax(self.softmax, self.hmatrix[self.trace_encoder[0:,self.t],self.t]) #softmax function over edge weights connected to current memory trace
        random_selection = np.random.choice(range(len(activation_weights)), p = activation_weights) #get destination of PS random walk
        self.percept_activations = np.zeros(np.shape(self.hmatrix)[0]) #reset activations
        #bias activation toward starting node of PS random walk
        self.percept_activations[self.trace_encoder[0:,self.t]] = [activation_weights[i] + self.focus*(1-activation_weights[i]) if i == random_selection else activation_weights[i] * (1-self.focus) for i in range(len(activation_weights))]

    def diffuse_activation(self):
        new_trace_activations = np.zeros(self.capacity)
        for i in range(np.shape(self.hmatrix)[0]):
            #edge weights are the product of the the edge h-values exponent (relevance), the activation of the trace to which the edge is connected (belief prior), and the excitation of the trace to which the edge is connected (sensory evidence)
            edge_weights = np.multiply(np.exp(self.hmatrix[i,self.trace_encoder[i,0:]]), self.trace_activations[self.trace_encoder[i,0:]], self.trace_excitations[self.trace_encoder[i,0:]])
            if len(edge_weights) > 0: #dont diffusion activation if there are no edges (there should not be to activation to diffuse)    
                edge_probs = _softmax(self.softmax, edge_weights)
                random_selection = np.random.choice(range(len(edge_weights)), p = edge_probs) #get destination of Projective Simulations
                diffusion_mass = _get_diffusion_mass(self.percept_activations[i], edge_probs, random_selection, self.focus)
                new_trace_activations[self.trace_encoder[i,0:]] = new_trace_activations[self.trace_encoder[i,0:]] + diffusion_mass
        self.trace_activations = new_trace_activations

    def predict(self):
        self.trace_activations = np.matmul(self.trace_activations, self.mmatrix) # because each row of the matrix has either zeros or a single 1, this just propogates attention forward. non-linear functions might be necessary for more complex mmatrix structures
        self.expectations = np.zeros(np.shape(self.hmatrix)[0]) + self.min_expectation #hmatrix rows correspond to perceptual representations
        for trace in range(self.capacity):
            action_modifiers = np.exp(self.action_encoder * (np.nanmean(self.valences) - self.valences[trace])) #used to scale priming of action representations as a function of trace valence
            self.expectations = self.expectations + self.trace_activations[trace] * np.multiply(self.trace_encoder[0:,trace], action_modifiers)

    def get_surprise(self, percept):
        not_action = np.invert(self.action_encoder) #used to exclude action representations from surprise computations
        return np.sum(np.where(percept[not_action], -np.log2(self.expectations[not_action]), -np.log2(1-self.expectations[not_action])))
            
def _get_diffusion_mass(activation, edge_probs, random_selection, focus):
    #gets the activation mass that diffuses along each edge connected to a perceptual representation. Note the different effect of the focus parameter if the edge was selected by PS random walk
    diffusions = [None] * len(edge_probs)
    for i in range(len(edge_probs)):
        if i == random_selection:
            diffusions[i] = activation * (edge_probs[i] + focus*(1-edge_probs[i]))
        else:
            diffusions[i] = activation * edge_probs[i] * (1 - focus)
    return(diffusions)
                

In [3]:
test = Episodic_Memory(num_actions = 2, focus = 0.5)
test.mmatrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
test.deliberate(np.array([1,0,1,0]))
test.expectations

array([nan, nan, nan, nan])

In [5]:
test.surprise

6.658355759469839

In [6]:
test.trace_encoder

array([[ True, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [ True, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False]])